# import dependecies

In [5]:
import cv2
import os
import numpy as np
import uuid
from pathlib import Path
from sklearn.model_selection import train_test_split
import json

# DataSet Generation

In [10]:
def generate_dataset(name):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            unique_filename = f"{name}_{str(uuid.uuid4())}.jpg"
            file_name_path = os.path.join("data", unique_filename)
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(1) == 13 or img_id == 200: # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

# Example usage
generate_dataset("Mahesh_Semwal")

In [ ]:
# Function to detect faces and crop them from an image
def face_cropped(img, face_classifier):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    cropped_faces = []
    for (x, y, w, h) in faces:
        cropped_faces.append(img[y:y+h, x:x+w])
    
    return cropped_faces

# Initialize the face cascade classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Directory containing images to process
data_dir = "data"

# Ensure the target directory exists
os.makedirs(data_dir, exist_ok=True)

# Process each image in the data directory
for filename in os.listdir(data_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        image_path = os.path.join(data_dir, filename)
        img = cv2.imread(image_path)
        
        # Crop faces from the image
        cropped_faces = face_cropped(img, face_classifier)
        
        if cropped_faces:
            # Save each cropped face
            for i, face in enumerate(cropped_faces):
                # Generate a unique filename using UUID
                unique_filename = f"{filename.split('.')[0]}_face_{i}_{str(uuid.uuid4())}.jpg"
                save_path = os.path.join(data_dir, unique_filename)
                
                # Save the cropped face
                cv2.imwrite(save_path, face)
                print(f"Saved {save_path}")
        
        # Delete the original image
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"Deleted {image_path}")
        else:
            print(f"Original image {image_path} not found.")

print("Face cropping and deletion completed for all images in 'data' directory.")

# Training Classifier

In [6]:
import numpy as np
from PIL import Image
import os
import cv2
import json

In [14]:
def train_classifier(data_dir):
    # Load existing label map if it exists
    if os.path.exists("label_map.json"):
        with open("label_map.json", "r") as f:
            label_map = json.load(f)
        current_id = max(label_map.values()) + 1
    else:
        label_map = {}
        current_id = 0
    
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
     
    faces = []
    labels = []

    for image_path in path:
        # Load image and convert to grayscale
        img = Image.open(image_path).convert('L')
        image_np = np.array(img, 'uint8')

        # Extract name from the filename
        filename = os.path.split(image_path)[1]
        name = filename.split('_')[0]

        # Assign a numeric label to each unique name
        if name not in label_map:
            label_map[name] = current_id
            current_id += 1

        label = label_map[name]
        
        faces.append(image_np)
        labels.append(label)
     
    labels = np.array(labels)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, labels)
    clf.write("classifier.xml")
    
    # Save the updated label map to a file
    with open("label_map.json", "w") as f:
        json.dump(label_map, f)

    print(f"Training completed. {len(label_map)} unique labels saved to 'label_map.json'.")


In [15]:
train_classifier("data")

Training completed. 2281 unique labels saved to 'label_map.json'.


# Real-Time Face Recognition

In [10]:
# Function to draw boundaries around detected faces and recognize them
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
         
        # Perform face recognition
        id_, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
         
        # Display recognized name if confidence is high
        if confidence > 70:
            name = reverse_label_map.get(id_, "UNKNOWN")
            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
     
    return img



In [16]:
# Initialize the face cascade and the LBPH face recognizer
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
clf = cv2.face.LBPHFaceRecognizer_create()

# Check if classifier file exists, if not, train a new one
if not os.path.exists("classifier.xml"):
    train_classifier("data")
else:
    clf.read('classifier.xml')

# Capture video from the webcam and perform real-time face recognition
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply the face detection and recognition
    frame = draw_boundary(frame, face_cascade, scaleFactor=1.1, minNeighbors=5, color=(255, 0, 0), clf=clf)
    
    # Display the result
    cv2.imshow("Face Detection", frame)
    
    # Exit on 'Enter' key press
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()


# Getting data from LFW

In [23]:
import os
import cv2
import uuid
from pathlib import Path

# Define the source directory and the target directory
source_dir = "lfw"
target_dir = "data"

# Ensure the target directory exists
os.makedirs(target_dir, exist_ok=True)

# Process each image in the dataset
for person_dir in Path(source_dir).iterdir():
    if person_dir.is_dir():
        person_name = person_dir.name
        for image_path in person_dir.iterdir():
            if image_path.is_file() and image_path.suffix in [".jpg", ".jpeg", ".png"]:
                img = cv2.imread(str(image_path))
                gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                unique_filename = f"{person_name}_{str(uuid.uuid4())}.jpg"
                file_name_path = os.path.join(target_dir, unique_filename)
                cv2.imwrite(file_name_path, gray_img)

print("LFW dataset processed and saved in 'data' directory.")

LFW dataset processed and saved in 'data' directory.
